In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [9]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
!python3 -m pip install 'kfp>=0.1.31' --quiet
!pip3 install tensorflow==1.14 --upgrade

Requirement already up-to-date: kfp in /opt/conda/lib/python3.6/site-packages (0.1.30)
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 109.2MB 279kB/s eta 0:00:01
    100% |████████████████████████████████| 3.2MB 7.2MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 25.5MB/s ta 0:00:01
    100% |████████████████████████████████| 491kB 25.5MB/s ta 0:00:01


    100% |████████████████████████████████| 583kB 23.7MB/s ta 0:00:01
fairing 0.5 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
  Found existing installation: setuptools 40.9.0
    Uninstalling setuptools-40.9.0:
      Successfully uninstalled setuptools-40.9.0
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow-estimator 1.13.0
    Uninstalling tensorflow-estimator-1.13.0:
      Successfully uninstalled tensorflow-estimator-1.13.0
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1
You are using pip version 19.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## KubeFlow Pipelines Serving Component
In this notebook, we will demo:

* Saving a Keras model in a format compatible with TF Serving
* Creating a pipeline to serve a trained model within a KubeFlow cluster

Reference documentation:
* https://www.tensorflow.org/tfx/serving/architecture
* https://www.tensorflow.org/beta/guide/keras/saving_and_serializing
* https://www.kubeflow.org/docs/components/serving/tfserving_new/

### Setup


In [28]:
# Set your output and project. !!!Must Do before you can proceed!!!
project =  'Your-Gcp-Project-ID'                          #'Your-GCP-Project-ID'
model_name = 'model-name'                                      # Model name matching TF_serve naming requirements       
import time
ts = int(time.time())
model_version = str(ts)                                            # Here we use timestamp as version to avoid conflict 
output = 'Your-Gcs-Path'     # A GCS bucket for asset outputs
KUBEFLOW_DEPLOYER_IMAGE = 'gcr.io/ml-pipeline/ml-pipeline-kubeflow-deployer:1.4.0'

In [30]:
model_path = '%s/%s' % (output,model_name)               
model_version_path = '%s/%s/%s' % (output,model_name,model_version)

### Load a Keras Model 
Loading a pretrained Keras model to use as an example. 

In [32]:
import tensorflow as tf

In [33]:
model = tf.keras.applications.NASNetMobile(input_shape=None,
                                   include_top=True,
                                   weights='imagenet',
                                   input_tensor=None,
                                   pooling=None,
                                   classes=1000)

Exception ignored in: <bound method _CheckpointRestoreCoordinator.__del__ of <tensorflow.python.training.tracking.util._CheckpointRestoreCoordinator object at 0x7f31dc197f98>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/training/tracking/util.py", line 244, in __del__
    .format(pretty_printer.node_names[node_id]))
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/training/tracking/util.py", line 93, in node_names
    path_to_root[node_id] + (child.local_name,))
  File "/opt/conda/lib/python3.6/site-packages/tensorflow/python/training/tracking/object_identity.py", line 76, in __getitem__
    return self._storage[self._wrap_key(key)]
KeyError: (<tensorflow.python.training.tracking.object_identity._ObjectIdentityWrapper object at 0x7f32aca065f8>,)


### Saved the Model for TF-Serve
Save the model using keras export_saved_model function. Note that specifically for TF-Serve the output directory should be structure as model_name/model_version/saved_model.

In [34]:
tf.keras.experimental.export_saved_model(model, model_version_path)

### Create a pipeline using KFP TF-Serve component


In [36]:
def kubeflow_deploy_op():
    return dsl.ContainerOp(
        name = 'deploy',
        image = KUBEFLOW_DEPLOYER_IMAGE,
        arguments = [
            '--model-export-path', model_path,
            '--server-name', model_name,
        ]
    )

In [37]:
import kfp
import kfp.dsl as dsl

# The pipeline definition
@dsl.pipeline(
    name='Sample model deployer',
    description='Sample for deploying models using KFP model serving component'
)
def model_server():
    deploy = kubeflow_deploy_op()

Submit pipeline for execution on Kubeflow Pipelines cluster

In [39]:
kfp.Client().create_run_from_pipeline_func(model_server, arguments={})

#vvvvvvvvv This link leads to the run information page. (Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working)